# doing more with data

  1. download the file
  2. unzip it
  3. read it and convert to pandas dataframe
  4. build a machine learning prediction engine


Download link: https://www.dropbox.com/s/u6rtu7a7clb4k3x/yelp_training_set_review.json.zip?dl=1


In [ ]:
import requests
import humanize
from __future__ import print_function
import zipfile
import pandas as pd
from matplotlib import pyplot
import seaborn as sns
from StringIO import StringIO
from lxml import etree

In [ ]:
humanize.naturalsize(10000000)

In [ ]:
file_url = "https://www.dropbox.com/s/u6rtu7a7clb4k3x/yelp_training_set_review.json.zip?dl=1"
r = requests.get(file_url, stream=True)
out = "yelp_training_set_review.json.zip"
out_size = 0
with open(out, "wb") as f:
    chunk_size = 10000000
    for chunk in r.iter_content(chunk_size=chunk_size):
        if chunk:
            out_size += chunk_size
            print("wrote {}".format(humanize.naturalsize(out_size)))
            f.write(chunk)
print("done")


In [ ]:
out = "yelp_training_set_review.json.zip"
zipf = zipfile.ZipFile(out)

In [ ]:
zipf.printdir()

In [ ]:
jsons = []
[jsons.append(x) for x in zipf.read('yelp_training_set_review.json').split("\n")[:20000]]
len(jsons)

In [ ]:
json_str = "[{}]".format(",".join(jsons))
df = pd.read_json(json_str)
df

In [ ]:
for classtype in ('funny', 'useful', 'cool'):
    df[classtype] = map(lambda votes: votes[classtype], df['votes'])
    

In [ ]:
df.drop(['business_id', 'review_id', 'type', 'user_id', 'votes'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
%matplotlib inline

sns.barplot(x=df.stars, y=df.useful+df.cool+df.funny, color="red")
sns.barplot(x=df.stars, y=df.cool+df.funny, color="yellow")
sns.barplot(x=df.stars, y=df.funny, color="blue")


In [ ]:
df_funny = df.loc[df['funny'] != 0]
df_not_funny = df.loc[df['funny'] == 0]
df_funny['funny'] = True
df_not_funny['funny'] = False

df_half_funny = pd.concat([df_funny, df_not_funny])
df_half_funny

In [ ]:
from sklearn.cross_validation import train_test_split
train, test, labels_train, labels_test = train_test_split(df_half_funny['text'].values,
                                                          df_half_funny['funny'].values,
                                                          test_size=0.20,
                                                          random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('count_vectorize', CountVectorizer(ngram_range=(1, 3))),
    ('tfidf_trandformer', TfidfTransformer()),
    ('classifier', RandomForestClassifier())
])

pipeline.fit(train, labels_train)




In [ ]:
pipeline.score(test, labels_test)

In [ ]:
from sklearn.metrics import confusion_matrix

pred_label_test = pipeline.predict(test)
confusion_matrix(labels_test, pred_label_test)

In [ ]:
pipeline.predict_proba(["I went to the cheese shop and there wasn't any cheese",])

In [ ]:
scores = pipeline.predict_proba(test)
scores

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(labels_test, [x[0]*-1 for x in scores])
fpr, tpr

In [ ]:
plt = pyplot
plt.figure()
plt.plot(fpr, tpr, label="Roc curve (area = %0.2f)" % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], "k--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("false Positive Rate")
plt.ylabel("true Positive Rate")
plt.title("Receiver operating characteristic")
plt.legend(loc="lower right")
plt.show()

# On Your own

Do the same thing with the NHTSA. Consider building a prediction based off if the report was labeled with a injury. There is an injury column in the data. 


In [ ]:
r = requests.get("https://www.dropbox.com/s/fjwu5iassqtsnyo/nhtsa_as_xml.zip?dl=1")
zipf = zipfile.ZipFile(StringIO(r.content))

In [ ]:
injured_file = zipf.read("nhtsa_injured.xml")
not_injured_file = zipf.read("nhtsa_not_injured.xml")

In [ ]:
itree = etree.parse(StringIO(injured_file))
nitree = etree.parse(StringIO(not_injured_file))

In [ ]:
data = zip(itree.xpath("//rows/row/@c2"), itree.xpath("//rows/row/@c8")) + zip(nitree.xpath("//rows/row/@c2"), nitree.xpath("//rows/row/@c8"))

In [ ]:
len(data)